# Convolutional Networks

In [ ]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import numpy as np

**Convolutional networks** also known as CNNs, are specialized neural networks for processing data that has a known grid-like topology. Convolutional networks are simply neural networks that use convolution in place of general matrix multiplication in at least one of their layers. A **convolution** is a specialized kind of liner operation and this lesson will go in depth as to what exactly a convolution is. Convolutional neural networks are very commonly used for image recognition.

## Invariance

CNNs systematize the idea of spatial invariance and exploit it to learn useful representations with fewer parameters. Spatial invariance is the idea that if we are looking for a certain feature in an image, it doesn't matter where in the image that feature is located. For example, if we are looking for a cat in an image, it doesn't matter where in the image the cat is located. This is a very useful property for image recognition because it allows us to use the same weights to detect the same feature in different parts of the image.

There a couple of important design considerations we should take in:
1. In earliest layers, the network should respond similarly to the same feature regardless of where it appears in the image. This is called **translation invariance**.
2. The earliest layers should focus on local regions without regard for contents elsewhere in the image. This is called **locality**. Eventually the network should be able to combine local features into global features.
3. The deeper layers should be able to capture longer-range features of the image, in a way similar to higher level vision in nature.


## Motivation

Convolutions leverage three very important conepts that can help to improve machine learning systems:

1. Sparse interactions
2. Paratemer sharing
3. Equivariant representations

Moreover, convolution provides a means for working with inputs of variable size. This is very useful for image recognition because images can be of different sizes.

### Sparse Interactions

**Sparse interactions** (also referred to as sparse connectivity or sparse weights) are a way to reduce the number of parameters in a model. In a fully connected network, each neuron in a layer is connected to every neuron in the previous layer. This means that the number of parameters in a fully connected network grows quadratically with the number of neurons in a layer. In a convolutional network, each neuron is only connected to a few nearby neurons in the previous layer. This means that the number of parameters in a convolutional network grows linearly with the number of neurons in a layer. This is a huge advantage because it allows us to build much deeper networks.


Sparse interactions are accomplished by making the kernel smaller than the input. This means that we need to store fewer paramters, which both reduces the memory requireemnts of the model and imrpoves its statistical efficiency. It also means that computing outputs requires fewer parameters which reduces the memory requirements as well as the computational requirements.

### Parameter Sharing

**Parameter sharing** is the process of using the same parameter for more than one function in a model. Typically, traditional neural networks have each element of the weight matrix being used exactly once when computing the output of a layer. The parameter sharing used by convolution operation means that rather thna learning a separate set of parameters for every location, we only learn one set. This doesn't reduce the runtime of forward propogation but does reduce the memory requirements of the model and improves its statistical efficiency.

### Equivariant Representations

In the case of convolution, the particular form of parameter sharing causes the layer to have a property called **equivariance** to translation. Equivariance means that if we translate the input, the output gets translated the same amount. This is a very useful property for image recognition because it allows us to use the same weights to detect the same feature in different parts of the image.

## Convolutions

In math, a convolution between two functions is:

$$ (f * g)(t) = \int_{-\infty}^{\infty} f(\tau)g(t-\tau) d\tau $$

The convolution operation is typically denoted with an asterisk as shown above. In CNN terms, the first argument which here is $f$ is the input and the second is what we call the kernel. The output of this is sometimes referred to as the feature map. The convolution operation is a linear operation that preserves the order of the inputs. The convolution operation is commutative, meaning that $f * g = g * f$. The convolution operation is also associative, meaning that $f * (g * h) = (f * g) * h$. The convolution operation is also distributive, meaning that $f * (g + h) = f * g + f * h$.

The convolution operation measures the overlap between $f$ and $g$ when one function is "flipped" and shifted by x. The convolution operation is used in signal processing to measure the similarity between two signals that are shifted by some amount. The convolution operation is also used in probability theory to measure the probability of two independent events occurring at the same time.

![Convolutional Neural Network](https://miro.medium.com/v2/resize:fit:720/1*32zCSTBi3giSApz1oQV-zA.gif)

Whenever we have discrete objects, we can turn the integrals into sums. For instance, the convolution of two vectors $f$ and $g$ is:

$$ (f * g)[n] = \sum_{m=-\infty}^{\infty} f[m]g[n-m] $$

For two-dimensional images, the convolution operation is:

$$ 
(f * g)[i,j] = \sum_{m} \sum_{n} f[m,n]g[i-m,j-n] 
$$

Convolutional is commutative meaning that we can also write:

$$
(f * g)[i,j] = \sum_{m} \sum_{n} f[i-m,j-n]g[m,n]
$$

## Cross Correlation

**Cross-correlation** is also another thing that is commonly applied. Cross-correlation is the same as convolution except that the kernel is not flipped. So this means that it takes the formula:

$$
(f * g)[i,j] = \sum_{m} \sum_{n} f[m,n]g[i+m,j+n]
$$

Sometimes this is used interchangeably so pay attention to the context and pay attention to when we say the kernel is flipped.

Below is a two-dimensional tensor with a height of 3 and width of 3. The kernel is a 2x2 matrix. The shape of the kernel window (or convolutional window) is given by the height and width of the kernel. The kernel window is slid across the input tensor and the dot product is taken between the kernel and the input tensor at each position. The output of this is a 2x2 matrix.

![Convolutional Neural Network](https://d2l.ai/_images/correlation.svg)

In the 2-d cross-correlation operation, we begin the convolution window positioned at the top-left corner of the input tensor and slide it across the input tensor, both from left to right and top to bottom. When the convolution window slides to a certain position, the input sub-tensors contained in that window and the kernel tensor are multiplied (element-wise) and the resulting tensor is summed up yielding a single scalar value. This result gives the value of the output tensor at the corresponding location. We will refer to this single scalar as the **element** of the output tensor.

$$
0 \times 0 + 1 \times 1 + 3 \times 2 + 4 \times 3 = 19
$$

$$
1 \times 0 + 2 \times 1 + 4 \times 2 + 5 \times 3 = 25
$$

$$
3 \times 0 + 4 \times 1 + 6 \times 2 + 7 \times 3 = 37
$$

$$
4 \times 0 + 5 \times 1 + 7 \times 2 + 8 \times 3 = 43
$$

The output size of the convolution is given by the input size $n_h \times n_w$ minus the size of the convolution kernel $k_h \times k_w$ plus 1. This is because we need to have enough space to fit the convolution kernel. So the output size is:

$$
(n_h - k_h + 1) \times (n_w - k_w + 1)
$$

In the following code, we showcase how to implement a cross-correlation function to transform the input tensor and kernel into an output. The first code implements the cross-correlation function using PyTorch, and the second code implements the cross-correlation function using NumPy.

In [ ]:
def corr2d(X, K):
    """Compute 2D cross-correlation."""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i: i + h, j: j + w] * K).sum()
    return Y

In [ ]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

In [ ]:
def corr2d(X, K):  #@save
    """Compute 2D cross-correlation."""
    h, w = K.shape
    Y = np.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y


In [ ]:
X = np.array([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = np.array([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

Images are digitally stored as huge matrices of numbers where each number corresponds to the brightness of a signle pixel. Below, the number 9 is showcased as a matrix. 

In [ ]:
nineImg = np.array([[0, 0, 0, 0, 0, 0, 0, 0],
                 [0, 0, 0, 255, 255, 0, 0, 0],
                 [0, 0, 255, 0, 0, 255, 0, 0],
                 [0, 0, 255, 0, 0, 255, 0, 0],
                 [0, 0, 0, 255, 255, 255, 0, 0],
                 [0, 0, 0, 0, 0, 255, 0, 0],
                 [0, 0, 255, 255, 255, 0, 0, 0],
                 [0, 0, 0, 0, 0, 0, 0, 0]
                 ])
ax = plt.gca()
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
plt.imshow( nineImg )
plt.show()

Next, let's see what happens when we add a convolution to it:

In [ ]:
straight = np.array([[0, 1, 0], [0, 1, 0], [0, 1, 0]])
straightNine = corr2d(nineImg, straight)

In [ ]:
ax = plt.gca()
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
plt.imshow(straightNine)
plt.show()

Here we can see that this straight upwards line makes the vertical edges much more prominent.

**Convolutional layers** are the basic building blocks of CNNs. A convolutional layer consists of one or more filters that are convolved with the input to produce an output. The output of a convolutional layer is often referred to as a feature map. The filters are also called convolutional kernels. The filters are typically small spatially (e.g. 3x3 or 5x5 pixels) but extend through the full depth of the input volume. For example, if the input is a color image with a depth of 3, then the filters are 3-dimensional with a depth of 3. The filters are learned during the training process. The filters are typically initialized randomly and then updated during training using backpropagation.

The parameters of the convolutional layer are the kernel and the scalar bias. When training models based on convolutional layers, we typically initialize the kernels randomly. Here is the code for a convolutional layer in PyTorch:

In [ ]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

We refer to a convolutional layer with a $h \times w$ convolution kernel simply as a $h \times w$ convolutional layer where the height and width of the convolution kernel are $h$ and $w$ respectively.

## Padding and Stride

When we do convolutions, the image continues to get smaller and smaller, however, this eventually has to stop as we will run out of pixels to apply the convolution to. Padding and stride give us more control over the size of the output.

### Padding

As described prior, we lose pixels on the ends of our image as we add convolutions. To reduce the amount of pixels we lose, we can add padding. Padding is a technique where we add pixels of some value, usually 0, around the edges of the input image. Below is an example of a 1 pixel padding:

![Convolutional Neural Network](https://d2l.ai/_images/conv-pad.svg)

If we add a total of $p_h$ rows of padding and a total of $pw$ columns of padding, the output shape will be:

$$
(n_h - k_h + p_h + 1) \times (n_w - k_w + p_w + 1)
$$

This means that the output height and width will increase by $p_h$ and $p_w$ respectively.

In machine learning, it's highly common that we use convolution kernels with odd height and width values to preserve dimensionality while padding with the same number of rows on top and bottom, and the same number of columns on left and right.

Below is an example of padding in PyTorch:

In [ ]:
def comp_conv2d(conv2d, X):
    X = X.reshape((1, 1) + X.shape)
    Y = conv2d(X)
    return Y.reshape(Y.shape[2:])

In [ ]:
X = torch.rand(size=(8, 8))
conv2d = nn.LazyConv2d(1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, X).shape

### Stride

When we do convolutions, we slide the kernel over the image one pixel at a time. The amount we slide the kernel each time is called the **stride**. So far, we have used a stride of 1, but we can use a larger stride to skip pixels and make the output smaller. Below is an example of a stride of 3:

![Convolutional Neural Network](https://d2l.ai/_images/conv-stride.svg)

Stride is a parameter that determines how much we slide the kernel at each step. If the stride is 1, then we move the kernel one pixel at a time. If the stride is 2, then we move the kernel 2 pixels at a time and so on. Above the stride of 3 is showcased.

When we use strides, the height and width of the output shape are:

$$
\left\lfloor \frac{n_h - k_h + p_h + s_h}{s_h} \right\rfloor \times \left\lfloor \frac{n_w - k_w + p_w + s_w}{s_w} \right\rfloor

$$

Below is an example in PyTorch:

In [ ]:
conv2d = nn.LazyConv2d(1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape

In [ ]:
conv2d = nn.LazyConv2d(1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
comp_conv2d(conv2d, X).shape

## Multiple Input and Multiple Output Channels

## Pooling